In [1]:
import pandas as pd

In [2]:
basics = pd.read_csv('imdb_dataset/title.basics.tsv', sep='\t', low_memory=False, index_col='tconst')
cast = pd.read_csv('imdb_dataset/title.principals.tsv', sep='\t', low_memory=False, index_col='tconst')
crew = pd.read_csv('imdb_dataset/title.crew.tsv', sep='\t', low_memory=False, index_col='tconst')
names = pd.read_csv('imdb_dataset/name.basics.tsv', sep='\t', low_memory=False)
ratings = pd.read_csv('imdb_dataset/title.ratings.tsv', sep='\t', low_memory=False, index_col='tconst') 

In [ ]:
movies = basics.loc[basics['titleType'] == 'movie']
movies = movies.loc[movies['isAdult'] == '0']
movies = movies.loc[movies['runtimeMinutes'] != r'\N']
movies = movies.loc[movies['genres'] != r'\N']
movies = movies.drop(columns=['titleType', 'primaryTitle', 'isAdult', 'endYear'])

In [ ]:
ratings = ratings.loc[ratings.index.isin(movies.index)]
crew = crew.loc[crew.index.isin(movies.index)]
cast = cast.loc[cast.index.isin(movies.index)]
cast = cast.loc[(cast['category'] == 'actor') | (cast['category'] == 'actress')]
cast = cast.groupby('tconst')['nconst'].apply(lambda x: ', '.join(x))

In [ ]:
movies = pd.concat([movies, ratings, crew, cast], axis='columns', sort=False)
movies.dropna(inplace=True)
movies = movies.loc[movies['directors'] != r'\N']
movies = movies.loc[movies['writers'] != r'\N']
movies = movies.loc[movies['nconst'] != r'\N']
# movies['numVotes'].describe(): 50% == 112.0
movies = movies.loc[movies['numVotes'] > 112]

In [96]:
code_to_name = dict(zip(names['nconst'], names['primaryName']))

def replace_codes_with_names(cell):
    if pd.isna(cell):
        return cell
    parts = [p.strip() for p in str(cell).split(',')]
    replaced = []
    for part in parts:
        if part == '':
            continue
        nome = code_to_name.get(part, part)
        replaced.append(nome)
    return ', '.join(replaced)

movies.loc[:,'directors'] = movies['directors'].apply(replace_codes_with_names)
movies.loc[:,'writers'] = movies['writers'].apply(replace_codes_with_names)
movies.loc[:,'nconst'] = movies['nconst'].apply(replace_codes_with_names)

In [98]:
movies.to_csv('movies.csv')